# Part 3: Unbiased Evaluation using a New Test Set

In this part, we are given a new test set (`/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv`). We can now take advantage of the entire smart sample that we created in Part I. 

* Retrain a pipeline using the optimal parameters that the pipeline learned. We don't need to repeat GridSearch here. 

## Import modules as needed

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.covariance import EllipticEnvelope

## Load smart sample and the best pipeline from Part II

In [2]:
sampled_X, sampled_y, model = joblib.load('sample-data-v1.pkl')
pipe2, model_grid2 = joblib.load('data-part-3.pkl')


##  Retrain a pipeline using the full sampled training data set

Use the full sampled training data set to train the pipeline.

In [3]:
# Add code below this comment  (Question #E301)
# ----------------------------------

trained_model = model_grid2.fit(sampled_X, sampled_y)


### Save the trained model with the pickle library.

In [4]:
# Add code below this comment  
# -----------------------------

joblib.dump(trained_model, 'trained-model.pkl')


['trained-model.pkl']


## Load the Testing Data and evaluate your model

 * `/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv`
 
* We need to preprocess this test data (follow the steps similar to Part I)
* If we have fitted any normalizer/standardizer in Part 2, then we have to transform this test data using the fitted normalizer/standardizer

In [5]:
# Preprocess the given test set  (Question #E302)
# ----------------------------------

DATASET = '/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv'
dataset = pd.read_csv(DATASET).sample(frac = 1).reset_index(drop=True)
dataset.head().transpose()

dataset.info()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242076 entries, 0 to 242075
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   sku                242076 non-null  object 
 1   national_inv       242075 non-null  float64
 2   lead_time          227351 non-null  float64
 3   in_transit_qty     242075 non-null  float64
 4   forecast_3_month   242075 non-null  float64
 5   forecast_6_month   242075 non-null  float64
 6   forecast_9_month   242075 non-null  float64
 7   sales_1_month      242075 non-null  float64
 8   sales_3_month      242075 non-null  float64
 9   sales_6_month      242075 non-null  float64
 10  sales_9_month      242075 non-null  float64
 11  min_bank           242075 non-null  float64
 12  potential_issue    242075 non-null  object 
 13  pieces_past_due    242075 non-null  float64
 14  perf_6_month_avg   242075 non-null  float64
 15  perf_12_month_avg  242075 non-null  float64
 16  lo

In [6]:
del_cols = ['sku']
dataset.drop(del_cols, axis=1, inplace=True)

dataset.dropna()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,207.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,...,0.0,0.96,0.98,0.0,No,No,No,Yes,No,No
2,3.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.73,0.79,0.0,Yes,No,No,Yes,No,No
3,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.73,0.78,0.0,No,No,No,Yes,No,No
4,5.0,4.0,0.0,3.0,6.0,12.0,0.0,2.0,3.0,3.0,...,0.0,0.96,0.91,0.0,No,No,No,Yes,No,No
5,109.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.99,0.98,0.0,No,No,No,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242071,49.0,12.0,1.0,7.0,31.0,61.0,3.0,21.0,56.0,85.0,...,0.0,0.99,0.99,0.0,No,No,No,Yes,No,No
242072,1.0,8.0,0.0,1.0,1.0,2.0,0.0,1.0,1.0,2.0,...,0.0,0.98,0.98,0.0,No,No,No,Yes,No,No
242073,6.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.82,0.81,0.0,No,No,No,Yes,No,No
242074,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.98,0.98,0.0,No,No,No,Yes,No,No


In [7]:
yes_no_columns = list(filter(lambda i: dataset[i].dtype!=np.float64, dataset.columns))
print(yes_no_columns)

# Add code below this comment  (Question #E102)
# ----------------------------------

print('potential_issue', dataset['potential_issue'].unique())
print('deck_risk', dataset['deck_risk'].unique())
print('oe_constraint', dataset['oe_constraint'].unique())
print('ppap_risk', dataset['ppap_risk'].unique())
print('stop_auto_buy', dataset['stop_auto_buy'].unique())
print('rev_stop', dataset['rev_stop'].unique())
print('went_on_backorder', dataset['went_on_backorder'].unique())


['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']
potential_issue ['No' 'Yes' nan]
deck_risk ['No' 'Yes' nan]
oe_constraint ['No' 'Yes' nan]
ppap_risk ['No' 'Yes' nan]
stop_auto_buy ['Yes' 'No' nan]
rev_stop ['No' 'Yes' nan]
went_on_backorder ['No' 'Yes' nan]


In [8]:
for column_name in yes_no_columns:
    mode = dataset[column_name].apply(str).mode()[0]
    print('Filling missing values of {} with {}'.format(column_name, mode))
    dataset[column_name].fillna(mode, inplace=True)
    
dataset['potential_issue'] = dataset['potential_issue'].map({'No':0, 'Yes':1})
dataset['deck_risk'] = dataset['deck_risk'].map({'No':0, 'Yes':1})
dataset['oe_constraint'] = dataset['oe_constraint'].map({'No':0, 'Yes':1})
dataset['ppap_risk'] = dataset['ppap_risk'].map({'No':0, 'Yes':1})
dataset['stop_auto_buy'] = dataset['stop_auto_buy'].map({'No':0, 'Yes':1})
dataset['rev_stop'] = dataset['rev_stop'].map({'No':0, 'Yes':1})
dataset['went_on_backorder'] = dataset['went_on_backorder'].map({'No':0, 'Yes':1})

Filling missing values of potential_issue with No
Filling missing values of deck_risk with No
Filling missing values of oe_constraint with No
Filling missing values of ppap_risk with No
Filling missing values of stop_auto_buy with Yes
Filling missing values of rev_stop with No
Filling missing values of went_on_backorder with No


In [9]:
dataset.isna().sum()

national_inv             1
lead_time            14725
in_transit_qty           1
forecast_3_month         1
forecast_6_month         1
forecast_9_month         1
sales_1_month            1
sales_3_month            1
sales_6_month            1
sales_9_month            1
min_bank                 1
potential_issue          0
pieces_past_due          1
perf_6_month_avg         1
perf_12_month_avg        1
local_bo_qty             1
deck_risk                0
oe_constraint            0
ppap_risk                0
stop_auto_buy            0
rev_stop                 0
went_on_backorder        0
dtype: int64

In [11]:
dataset_fillna = list(filter(lambda i: dataset[i].isna().sum() >= 1, dataset.columns))

for column_name in dataset_fillna:
    mode = dataset[column_name].apply(str).mode()[0]
    print('Filling missing values of {} with {}'.format(column_name, mode))
    dataset[column_name].fillna(mode, inplace=True)
    

Filling missing values of national_inv with 2.0
Filling missing values of lead_time with 8.0
Filling missing values of in_transit_qty with 0.0
Filling missing values of forecast_3_month with 0.0
Filling missing values of forecast_6_month with 0.0
Filling missing values of forecast_9_month with 0.0
Filling missing values of sales_1_month with 0.0
Filling missing values of sales_3_month with 0.0
Filling missing values of sales_6_month with 0.0
Filling missing values of sales_9_month with 0.0
Filling missing values of min_bank with 0.0
Filling missing values of pieces_past_due with 0.0
Filling missing values of perf_6_month_avg with 0.73
Filling missing values of perf_12_month_avg with 0.99
Filling missing values of local_bo_qty with 0.0


In [12]:
dataset.isna().sum()

national_inv         0
lead_time            0
in_transit_qty       0
forecast_3_month     0
forecast_6_month     0
forecast_9_month     0
sales_1_month        0
sales_3_month        0
sales_6_month        0
sales_9_month        0
min_bank             0
potential_issue      0
pieces_past_due      0
perf_6_month_avg     0
perf_12_month_avg    0
local_bo_qty         0
deck_risk            0
oe_constraint        0
ppap_risk            0
stop_auto_buy        0
rev_stop             0
went_on_backorder    0
dtype: int64

In [13]:
X_test = dataset.drop('went_on_backorder', axis=1)
y_test = dataset.went_on_backorder

processor = MinMaxScaler()
scaled_X = processor.fit_transform(X_test)

We can now predict and evaluate with the preprocessed test set. It would be interesting to see the performance with and without outliers removal from the test set. We can report confusion matrix, precision, recall, f1-score, accuracy, and other measures (if any). 

In [15]:
# Add code below this comment  (Question #E303)
# ----------------------------------


#without outlier removal
pred_y = trained_model.predict(X_test)

model_grid2.score(X_test, y_test)
print(classification_report(y_test, pred_y))
pd.DataFrame(confusion_matrix(y_test, pred_y))




              precision    recall  f1-score   support

           0       0.99      0.79      0.88    239388
           1       0.03      0.55      0.05      2688

    accuracy                           0.79    242076
   macro avg       0.51      0.67      0.47    242076
weighted avg       0.98      0.79      0.87    242076



,0,1
0,188705,50683
1,1213,1475


## Conclusion

## Reflect

Imagine you are data scientist that has been tasked with developing a system to save your 
company money by predicting and preventing back orders of parts in the supply chain.

Write a **brief summary** for "management" that details your findings, 
your level of certainty and trust in the models, 
and recommendations for operationalizing these models for the business.

# Save your notebook!
## Then `File > Close and Halt`